<a href="https://colab.research.google.com/github/AlexEskin/Templates/blob/master/Upload_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Init directory tree

In [0]:
!rm -rf crops_train
!rm -rf crops_test
!rm -rf maps_resized
!rm -rf examples

!mkdir maps_resized
!mkdir crops_train
!mkdir crops_test
!mkdir examples

In [0]:
import os
import cv2
import numpy as np
import os
from google.colab import files

WIDTH = 256
HEIGHT = 256
SOURCE = 'MapsExample'

In [0]:
def resizeImages():
    path_a = SOURCE
    path_b = 'maps_resized'

    filenames = []
    for r, d, f in os.walk(path_a):
        for file in f:
            if ".png" in file:
               filenames.append(file)
    print(filenames)
    images = []
    minSizeX = None
    minSizeY = None

    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        print(f_a)
        im_a = cv2.imread(f_a)
        if minSizeX != None:
            minSizeX = min(minSizeX,im_a.shape[0])
        else:
            minSizeX = im_a.shape[0]
        if minSizeY != None:
            minSizeY = min(minSizeY,im_a.shape[1])          
        else:
            minSizeY = im_a.shape[1]
    print(minSizeX,minSizeY)
    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        f_b = os.path.join(path_b, filename)

        im_a = cv2.imread(f_a)
        crop_img = im_a[0:minSizeX, 0:minSizeY]
        cv2.imwrite(f_b,crop_img)


In [0]:
def generateCrops():
	path_a = 'maps_resized'
	path_b = 'crops_train'
	path_c = 'crops_test'
	examples = 'examples'
	filenames = []
	for r, d, f in os.walk(path_a):
		for file in f:
			filenames.append(file)
	counter = 0
	for filename in filenames:
		print(filename)
		f_a = os.path.join(path_a, filename)
		im_a = cv2.imread(f_a)
		shape = im_a.shape
		examples_counter = 0
		for i in range(0,shape[0]-WIDTH-1,64):
			for j in range(0,shape[1]-HEIGHT-1,64):
				path = path_b
				if not counter%7:
					path = path_c				
				submap = os.path.join(path,str(counter)+".png")
				submap_image = im_a[i:i+HEIGHT,j:j+WIDTH]
				cv2.imwrite(submap,submap_image)
				if examples_counter < 32:
					expath = os.path.join(examples, filename)
					try:
						if not examples_counter:
							os.mkdir(expath)
					except Exception as e:
						pass
					exmap = os.path.join(expath,str(counter)+".png")
					cv2.imwrite(exmap,submap_image)
					examples_counter = examples_counter + 1
				counter += 1
		print(counter)

Upload files

In [0]:
!rm -rf MapsExample

In [16]:
!git clone https://github.com/AlexEskin/MapsExample.git

Cloning into 'MapsExample'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 55 (delta 0), reused 55 (delta 0), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [17]:
!ls MapsExample

map_10.png  map_19.png	map_27.png  map_35.png	map_43.png  map_5.png
map_11.png  map_1.png	map_28.png  map_36.png	map_44.png  map_6.png
map_12.png  map_20.png	map_29.png  map_37.png	map_45.png  map_7.png
map_13.png  map_21.png	map_2.png   map_38.png	map_46.png  map_8.png
map_14.png  map_22.png	map_30.png  map_39.png	map_47.png  map_9.png
map_15.png  map_23.png	map_31.png  map_3.png	map_48.png  README.md
map_16.png  map_24.png	map_32.png  map_40.png	map_49.png
map_17.png  map_25.png	map_33.png  map_41.png	map_4.png
map_18.png  map_26.png	map_34.png  map_42.png	map_50.png


In [42]:
resizeImages()
generateCrops()

['map_50.png', 'map_31.png', 'map_29.png', 'map_23.png', 'map_12.png', 'map_14.png', 'map_1.png', 'map_21.png', 'map_34.png', 'map_2.png', 'map_42.png', 'map_17.png', 'map_4.png', 'map_7.png', 'map_37.png', 'map_24.png', 'map_18.png', 'map_45.png', 'map_16.png', 'map_11.png', 'map_30.png', 'map_10.png', 'map_48.png', 'map_44.png', 'map_39.png', 'map_36.png', 'map_22.png', 'map_13.png', 'map_40.png', 'map_3.png', 'map_28.png', 'map_5.png', 'map_19.png', 'map_25.png', 'map_38.png', 'map_20.png', 'map_49.png', 'map_33.png', 'map_8.png', 'map_46.png', 'map_6.png', 'map_32.png', 'map_41.png', 'map_26.png', 'map_43.png', 'map_35.png', 'map_9.png', 'map_47.png', 'map_27.png', 'map_15.png']
MapsExample/map_50.png
MapsExample/map_31.png
MapsExample/map_29.png
MapsExample/map_23.png
MapsExample/map_12.png
MapsExample/map_14.png
MapsExample/map_1.png
MapsExample/map_21.png
MapsExample/map_34.png
MapsExample/map_2.png
MapsExample/map_42.png
MapsExample/map_17.png
MapsExample/map_4.png
MapsExample/

In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import random
from cv2 import imread

input_img = Input(shape=(256, 256, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (20, 20), activation='relu', padding='same')(input_img)
x = MaxPooling2D((4, 4), padding='same')(x)
x = Conv2D(8, (20, 20), activation='relu', padding='same')(x)
x = MaxPooling2D((4, 4), padding='same')(x)
x = Conv2D(8, (20, 20), activation='relu', padding='same')(x)
encoded = MaxPooling2D((4, 4), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 208-dimensional

x = Conv2D(8, (20, 20), activation='relu', padding='same')(encoded)
x = UpSampling2D((4, 4))(x)
x = Conv2D(8, (20, 20), activation='relu', padding='same')(x)
x = UpSampling2D((4, 4))(x)
x = Conv2D(16, (20, 20), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (20, 20), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (20, 20), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()
x_train = []
x_test = []

path_b = 'crops_train'
path_c = 'crops_test'

filenames = []
for r, d, f in os.walk(path_b):
    for file in f:
        filenames.append(file)

random.shuffle(filenames)
filenames = filenames[0:400]
for filename in filenames:
    f_a = os.path.join(path_b, filename)
    src  =imread(f_a)
    if len(src) > 3:
        x_train.append(src.astype(float)*(1/255.0)) 
x_train = np.array(x_train)

filenames = []
for r, d, f in os.walk(path_c):
    for file in f:
        filenames.append(file)

random.shuffle(filenames)
filenames = filenames[0:400]
print(len(filenames))
for filename in filenames:
    f_a = os.path.join(path_c, filename)
    src  =imread(f_a)
    if len(src) > 3:
        src = src.astype(float)*(1/255.0)
        x_test.append(src)

x_test = np.array(x_test)

autoencoder.fit(x_train, x_train,
                epochs=500,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 256, 256, 16)      19216     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 64, 8)         51208     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 16, 16, 8)         25608     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 8)           0   